In [ ]:
%load_ext Cython

In [ ]:
import pandas as pd
import numpy as np

y = np.random.randint(2, size=(5000, 1))
x = np.random.randint(10, size=(5000, 1))
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])

# Original Version

In [ ]:
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')]
    return result

In [ ]:
def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [ ]:
%%timeit
target_mean_v1(data, 'y', 'x')

1 loop, best of 3: 26.1 s per loop


In [ ]:
%%timeit
target_mean_v2(data, 'y', 'x')

1 loop, best of 3: 261 ms per loop


# Cython Version

In [ ]:
%%cython -a
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

import time
import numpy as np
cimport numpy as np
cimport cython
from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v3(data, y_name, x_name):
    X = data[x_name].values
    Y = data[y_name].values
    cdef Py_ssize_t N = X.shape[0] 
    v_dict = np.zeros(N, dtype=np.int)
    c_dict = np.zeros(N, dtype=np.int)
    cdef long[::1] value_dict = v_dict
    cdef long[::1] count_dict = c_dict

    cdef long[:] x_pt = X 
    cdef long[:] y_pt = Y
    cdef Py_ssize_t i
    cdef long x_id
    for i in range(N):
        x_id = x_pt[i]
        value_dict[x_id] += y_pt[i]
        count_dict[x_id] += 1
    
    result = np.zeros(N)
    cdef double[::1] result_pt = result
    for i in range(N): 
        x_id = x_pt[i]
        result_pt[i] = (value_dict[x_id] - y_pt[i]) / (count_dict[x_id] - 1)
    return result


In [ ]:
%%timeit
target_mean_v3(data, 'y', 'x')

The slowest run took 5.18 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 31.4 µs per loop


In [ ]:
result_2 = target_mean_v2(data, 'y', 'x')
result_3 = target_mean_v3(data, 'y', 'x')
diff = np.linalg.norm(result_2 - result_3)
print(diff)

0.0


In [ ]:
26.1/ (31.4 / 1e6)

831210.1910828026

# Cython with multiple threads

In [ ]:
%%cython --cplus
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

import numpy as np
import time
cimport numpy as np
cimport cython
from cython.parallel import prange

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef target_mean_v4(data, y_name, x_name):
    X = data[x_name].values
    Y = data[y_name].values
    cdef Py_ssize_t N = X.shape[0] 
    v_dict = np.zeros(N, dtype=np.int)
    c_dict = np.zeros(N, dtype=np.int)
    cdef long[::1] value_dict = v_dict
    cdef long[::1] count_dict = c_dict

    cdef long[:] x_pt = X 
    cdef long[:] y_pt = Y
    cdef Py_ssize_t i
    cdef long x_id
    for i in prange(N, nogil=True):
        x_id = x_pt[i]
        value_dict[x_id] += y_pt[i]
        count_dict[x_id] += 1
    
    result = np.zeros(N)
    cdef double[::1] result_pt = result
    for i in prange(N, nogil=True): 
        x_id = x_pt[i]
        result_pt[i] = (value_dict[x_id] - y_pt[i]) / (count_dict[x_id] - 1)
    return result

In [ ]:
%%timeit
target_mean_v4(data, 'y', 'x')

The slowest run took 31.02 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 279 µs per loop


- The performance got worse with multiple threads, and I guess this is due to the overhead introduced by threading, which has increased the runtime when the original code is efficient enough so there is little room for improvement.